In [1]:
import pandas as pd
from time import sleep
import os
from selenium import webdriver # for interacting with website
from selenium.webdriver.common.by import By
import re

In [2]:
def transcript2df(transcript):
    pattern = r'^(([0-9]{2}|[0-9]{1}):([0-9]{2}|[0-9]{1})|([0-9]{2}|[0-9]{1})):[0-9]{2}$'
    transcript_timestamps,transcript_dialog,prev = [],[],None
    for item in transcript:
        if re.match(pattern, item):
            transcript_timestamps.append(item)
            if prev == None: prev = item
            else: 
                transcript_dialog.append(prev)
                prev = item
        else:
            if re.match(pattern, prev):
                prev = item
            else:
                prev = prev + ' ' + item
    transcript_dialog.append(item)
    df = pd.DataFrame({'timestamp':transcript_timestamps, 
                   'text':transcript_dialog})    
    return df

In [5]:
def get_youtube_transcript(video_ids):
    frames = []
    driver = webdriver.Chrome()
    for video_id in video_ids:
        try:
            driver.get(f"https://www.youtube.com/watch?v={video_id}")
            sleep(4)
            driver.find_element(By.XPATH,'//*[@id="button-shape"]/button').click()
            sleep(2)
            driver.find_element(By.XPATH,'//*[@id="items"]/ytd-menu-service-item-renderer[2]/tp-yt-paper-item/yt-formatted-string').click()
            sleep(4)
            transcript_element = driver.find_element(By.XPATH,"//*[@id='body']/ytd-transcript-segment-list-renderer")
            transcript = transcript_element.text
            transcript = transcript.split('\n')
            transcript.pop(0)
            frames.append(transcript2df(transcript).assign(video_id=video_id))
        except:
            print(video_id)
    driver.quit()
    return pd.concat(frames,ignore_index=True)

In [4]:
df = pd.read_parquet('../scripts/data/scrap_oso_trava/playlist_videos.parquet')

In [6]:
result = get_youtube_transcript(df.id.unique()[20:25])
result

nYE_PI1g2G4
Zi298e7ohUw


,timestamp,text,video_id
0,0:00,el punto central de lo que tiene una filosofía...,fQYtqIELCVE
1,0:05,aprendido algo que es adaptación si tú no te a...,fQYtqIELCVE
2,0:11,llegué a japón pensando que japón ciudad a par...,fQYtqIELCVE
3,0:17,de crecimiento [Música],fQYtqIELCVE
4,0:31,hola y bienvenidos a un nuevo episodio de crac...,fQYtqIELCVE
...,...,...,...
2774,1:26:46,quieres recibir todos los bienes un correo muy...,7y2y9lTs5r0
2775,1:26:53,frases artículos cosas que me recomiendan mis ...,7y2y9lTs5r0
2776,1:27:00,creo que pueden ayudarte a tener una vida más ...,7y2y9lTs5r0
2777,1:27:07,de cracks es totalmente gratis y puedes recibi...,7y2y9lTs5r0


In [ ]:
result.to_parquet('../scripts/data/scrap_oso_trava/videos_s',)